In [ ]:
#Iniciando com a instalação da ADK
%pip -q install google-adk

In [ ]:
#Importação dos módulos genai e types
from google import genai
from google.genai import types

#Importação das classes Agent, Runner, InMemorySessionService e google_search
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

#Importação do módulo de datas, para trabalhar com datas
from datetime import date

#Importação do textwrap, para manipular quebra de linhas e formatar textos
import textwrap

#Importação das funções display e Markdown, para estilizações
from IPython.display import display, Markdown

#Importação da biblioteca request, para interação HTTP
import requests

#Importação do módulo warning, para controle de avisos e função pra ignorar avisos.
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Configura a API Key do Google Gemini

import os

from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Treinos --- #
##########################################

def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informações sobre treino para alunos do nível iniciante.",
        tools=[google_search],
        instruction="""
        Você é um assistente de pesquisas, que está auxiliando um aluno novato com exercícios físicos e prática de atividade física. Sua tarefa é usar a ferramenta de busca do google (google_search)para pesquisar por informações sobre treinos, atividades físicas, benefícios para a saúde através da atividade física e assuntos correlacionados ao mundo fitness, inclusive, dicas para o dia a dia saudável. Não seja prolixo, traga informações relevantes, de fácil compreensão e sempre traga informações com embassamento científico (não complique a cabeça do nosso aluno, lembre-se: ele é novato). Se um conteúdo buscado não tiver muita qualidade, ou seja, pouco acesso, baixa relevância e sem embasamento científico, ignore e busque algo melhor. Essas informações devem ser mostradas em português brasileiro (se tiver algo de outro país, faça a tradução para o português do Brasil para que possa ser utilizado), de no máximo 3 meses antes da data de hoje. Se limite a trazer até 5 tópicos no conteúdo buscado.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [ ]:
################################################
# --- Agente 2: Planejador de treinos --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em atividade física, mundo fitness, educação física e personal trainner. Com base na lista de lançamentos mais recentes e relevantes buscados, você deve:	Usar a ferramenta de busca do Google (google_search) para criar um plano de treinamentos e prática de atividade física, sobre quais são os pontos mais relevantes que poderiamos abordar compor um plano de treinamento de atividades físicas. Você também pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar. Ao final, você irá escolher de três até cinco práticas de atividades físicas, seus pontos mais relevantes, e um plano com esses tópicos a serem abordados em um planejamentpo de treinos que será desenvolvido posteriormente.
        """,
        description="Agente que planeja treinamentos",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Prescritor do Treinos --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um prescritor de treinos, especialista em mundo fitness e educação física. Você prescreve treinos para a empresa FitNerd, a maior rede de academias do Brasil. Utilize o material fornecido no plano de treino e os pontos mais relevantes fornecidos e, com base nisso, escreva um plano de treino que inclua a execução perfeita da atividade física e as melhores práticas a serem adotadas para este plano em um rascunho que será utilizado na academia FitNerd. O plano deve ser engajador, informativo, com linguagem simples e incluir de 3 até 5 exercícios.
            """,
        description="Agente redator de planos de treino para a academia FitNerd"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em academias, prática de atividades física, mundo fitness e educação física. Você está atendendo alunos do nível iniciante, logo, não possuem uma compreensão tão profunda sobre termos de academia e só estão iniciando na vida fitness. Revise o rascunho do plano de treinamento da academia "FitNerd" verificando clareza, concisão, correção e tom. Se o plano de treino estiver bom, responda apenas 'O plano está ótimo e pronto para ser gerado!'. Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de plano de treinamentos para a FitNerd."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de apoio ao aluno iniciante da FitNerd com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer gerar o seu treino: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
	print("Você esqueceu de digitar o tópico!")
else:
	print (f"Maravilha! Vamos então vamos criar o seu plano: {topico}")

lancamentos_buscados = agente_buscador(topico, data_de_hoje)
print("\n--- Resultados do Agente 1 (Buscador) ---\n")
display(to_markdown(lancamentos_buscados))
print("----------------------------------------------------")

plano_de_post = agente_planejador(topico, lancamentos_buscados)
print("\n--- Resultado do Agente 2 (Planejador) ---\n")
display(to_markdown(plano_de_post))
print("----------------------------------------------------")

rascunho_de_post = agente_redator(topico, plano_de_post)
print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
display(to_markdown(rascunho_de_post))
print("--------------------------------------------------------------")

post_final = agente_revisor(topico, rascunho_de_post)
print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
display(to_markdown(post_final))
print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de apoio ao aluno iniciante da FitNerd com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer gerar o seu treino: Treinos para multiplas articulações
Maravilha! Vamos então vamos criar o seu plano de treinamento Treinos para multiplas articulações

--- Resultados do Agente 1 (Buscador) ---



> Olá! Treinos multiarticulares são ótimos para iniciantes, pois trabalham vários grupos musculares ao mesmo tempo, o que aumenta o gasto calórico e a força geral. Aqui estão alguns exercícios que você pode incluir no seu treino:
> 
>  
> Com certeza! Exercícios multiarticulares são excelentes para quem está começando. Eles trabalham vários músculos ao mesmo tempo, otimizando seu tempo e aumentando a eficiência do treino.
> 
> **Benefícios dos exercícios multiarticulares:**
> 
> 1.  **Otimização do tempo:** Trabalham diversos grupos musculares simultaneamente.
> 2.  **Maior gasto calórico:** Ideais para quem busca perder peso, pois exigem mais energia do corpo.
> 3.  **Aumento da força e massa muscular:** Promovem microlesões que levam à hipertrofia e maior liberação hormonal.
> 4.  **Melhora da coordenação motora:** Exigem integração entre diferentes partes do corpo.
> 5.  **Funcionalidade:** Preparam o corpo para atividades do dia a dia, fortalecendo os músculos de forma integrada.
> 
> **Exemplos de exercícios multiarticulares para iniciantes:**
> 
> 1.  **Agachamento:** Trabalha pernas, glúteos e core. Mantenha os pés afastados na linha dos ombros e a postura ereta.
> 2.  **Supino:** Trabalha peito, ombros e tríceps. Pode ser feito com barra ou halteres.
> 3.  **Remada:** Trabalha costas e bíceps. A remada curvada com halteres é uma boa opção.
> 4.  **Flexão:** Trabalha peito, ombros, costas e tríceps. Comece com apoio nos joelhos se necessário.
> 5.  **Levantamento Terra:** Trabalha a parte inferior e superior do corpo. Essencial para fortalecer a cadeia posterior.
> 
> **Dicas:**
> 
> *   **Aquecimento:** Sempre aqueça antes de começar o treino.
> *   **Postura:** Mantenha a postura correta durante os exercícios para evitar lesões.
> *   **Progressão:** Aumente a carga gradualmente, conforme for ganhando força.
> *   **Core:** Ative a musculatura do abdômen para estabilizar o quadril e proteger a coluna.
> 
> Lembre-se de que é fundamental procurar orientação de um profissional de educação física para montar um treino adequado às suas necessidades e objetivos.
> 


----------------------------------------------------

--- Resultado do Agente 2 (Planejador) ---



> Com certeza! Com base nas informações fornecidas e em pesquisas adicionais, aqui está um plano de treinamento com foco em exercícios multiarticulares, ideal para iniciantes, detalhando os pontos mais relevantes a serem abordados em um planejamento de treinos posterior:
> 
> 
> ### **Plano de Treinamento para Iniciantes com Foco em Exercícios Multiarticulares**
> 
> **Objetivo:** Desenvolver força geral, aumentar o gasto calórico, melhorar a coordenação motora e preparar o corpo para atividades diárias, utilizando exercícios que trabalham múltiplos grupos musculares simultaneamente.
> 
> **Princípios Fundamentais:**
> 
> 1.  **Seleção de Exercícios:**
> 
>     *   **Exercícios Essenciais:** Escolher de 3 a 5 exercícios multiarticulares que cubram os principais grupos musculares.
>         *   **Exemplos:** Agachamento, supino, remada, flexão e levantamento terra (com adaptações para iniciantes).
> 
> 2.  **Técnica Correta e Postura:**
> 
>     *   **Demonstração e Correção:** Garantir que o praticante aprenda a execução correta de cada exercício desde o início.
>         *   **Foco:** Alinhamento postural, controle do movimento e ativação correta dos músculos envolvidos.
>     *   **Progressão:** Começar com cargas leves ou sem carga (utilizando o peso corporal) para dominar a técnica.
> 
> 3.  **Aquecimento e Alongamento:**
> 
>     *   **Aquecimento:** Preparar o corpo para o exercício com atividades cardiovasculares leves (5-10 minutos) e mobilidade articular.
>     *   **Alongamento:** Incluir alongamentos dinâmicos antes do treino e estáticos após o treino para melhorar a flexibilidade e reduzir o risco de lesões.
> 
> 4.  **Progressão Gradual da Carga:**
> 
>     *   **Aumento Progressivo:** Aumentar a carga (peso) gradualmente, conforme o praticante ganha força e controle.
>         *   **Atenção:** Priorizar a manutenção da técnica correta ao aumentar a carga.
>     *   **Variação:** Ajustar o número de repetições e séries para otimizar o estímulo muscular.
> 
> 5.  **Ativação do Core:**
> 
>     *   **Estabilização:** Ensinar o praticante a ativar a musculatura do core (abdômen e região lombar) para estabilizar o tronco durante os exercícios.
>         *   **Importância:** Proteção da coluna vertebral e melhoria da eficiência dos movimentos.
>     *   **Exercícios de Core:** Incluir exercícios específicos para o core como prancha e exercícios de estabilização.
> 
> 6.  **Descanso e Recuperação:**
> 
>     *   **Intervalo:** Respeitar os intervalos de descanso entre as séries para permitir a recuperação muscular.
>     *   **Sono:** Garantir uma boa noite de sono para otimizar a recuperação e o crescimento muscular.
>     *   **Nutrição:** Orientar sobre a importância de uma dieta equilibrada para fornecer os nutrientes necessários para a recuperação e o crescimento muscular.
> 
> 7.  **Importância do Acompanhamento Profissional:**
> 
>     *   **Acompanhamento Contínuo:** Enfatizar a necessidade de acompanhamento de um profissional de educação física para ajustar o treino, corrigir a técnica e prevenir lesões.
>     *   **Personalização:** Adaptar o plano de treino às necessidades e objetivos individuais do praticante.
>     *   **Motivação:** O profissional pode fornecer suporte e motivação para manter o praticante engajado no programa de treinamento.
> 
> ### **Exemplos de Exercícios Multiarticulares para Iniciantes:**
> 
> 1.  **Agachamento:**
> 
>     *   **Foco:** Pernas, glúteos e core.
>     *   **Execução:** Pés afastados na largura dos ombros, agachar como se fosse sentar em uma cadeira, mantendo a postura ereta e o core ativado.
>     *   **Variações:** Agachamento livre, agachamento com peso corporal, agachamento com halteres.
> 2.  **Supino:**
> 
>     *   **Foco:** Peito, ombros e tríceps.
>     *   **Execução:** Deitar em um banco, segurar a barra ou halteres com as mãos na largura dos ombros, abaixar o peso até o peito e empurrar de volta para cima.
>     *   **Variações:** Supino reto com barra, supino com halteres, supino inclinado.
> 3.  **Remada:**
> 
>     *   **Foco:** Costas e bíceps.
>     *   **Execução:** Inclinar o tronco para frente, segurar halteres ou barra, puxar o peso em direção ao abdômen, mantendo as costas retas.
>     *   **Variações:** Remada curvada com halteres, remada com barra, remada em máquina.
> 4.  **Flexão:**
> 
>     *   **Foco:** Peito, ombros, tríceps e core.
>     *   **Execução:** Mãos no chão na largura dos ombros, corpo reto, abaixar o corpo até o peito quase tocar o chão e empurrar de volta para cima.
>     *   **Variações:** Flexão com apoio nos joelhos (para iniciantes), flexão tradicional.
> 5.  **Levantamento Terra (Adaptado):**
> 
>     *   **Foco:** Cadeia posterior (isquiotibiais, glúteos, lombar) e core.
>     *   **Execução:** Pés afastados na largura dos ombros, agachar para pegar a barra ou halteres, mantendo as costas retas, levantar o peso estendendo as pernas e o tronco.
>     *   **Variações:** Levantamento terra com halteres, levantamento terra com barra (carga leve).
> 
> ### **Plano de Treino Sugerido (Exemplo):**
> 
> *   **Frequência:** 2-3 vezes por semana, com descanso de pelo menos um dia entre os treinos.
> *   **Estrutura:**
>     *   Aquecimento: 5-10 minutos de cardio leve e mobilidade articular.
>     *   Treino:
>         *   Agachamento: 3 séries de 8-12 repetições.
>         *   Supino: 3 séries de 8-12 repetições.
>         *   Remada: 3 séries de 8-12 repetições.
>         *   Flexão: 3 séries até a falha (ou com apoio nos joelhos).
>         *   Prancha: 3 séries, mantendo a posição por 30-60 segundos.
>     *   Alongamento: 5-10 minutos de alongamentos estáticos.
> *   **Progressão:** Aumentar a carga ou o número de repetições a cada semana, conforme a capacidade do praticante.
> 
> **Observações Finais:**
> 
> *   **Individualização:** Este plano é um exemplo e deve ser adaptado às necessidades e objetivos individuais de cada praticante, com a orientação de um profissional de educação física.
> *   **Consistência:** A chave para o sucesso é a consistência e a progressão gradual.
> *   **Segurança:** Priorizar sempre a técnica correta para evitar lesões.
> 


----------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> ## Rascunho - Plano de Treino FitNerd: Multiarticulares para Iniciantes
> 
> **Objetivo:** Boas-vindas ao mundo do movimento eficiente! Este treino vai te dar força, queimar calorias e te preparar para o dia a dia, tudo em um só pacote!
> 
> **Por que Multiarticulares?** 💪
> 
> Imagine um exercício que trabalha vários músculos ao mesmo tempo. É tipo fazer um "combo" de benefícios:
> 
> *   **Mais resultados em menos tempo:** Ideal para quem tem a agenda apertada.
> *   **Corpo mais forte e funcional:** Melhora a coordenação e te deixa pronto para qualquer desafio.
> *   **Queima de calorias turbinada:** Acelera o metabolismo e te ajuda a alcançar seus objetivos.
> 
> **Importante:** Se você for novo por aqui ou estiver voltando aos treinos, este plano é perfeito para começar com o pé direito!
> 
> **O Plano de Ataque:**
> 
> Faremos um circuito com foco em 4 exercícios chave, priorizando a forma correta e a adaptação ao seu ritmo.
> 
> **1. Aquecimento:** (5 minutos)
> 
> *   **Cardio Leve:** Uma caminhada na esteira ou polichinelos para aquecer o corpo.
> *   **Mobilidade:** Gire os braços, pernas e tronco para preparar as articulações.
> 
> **2. Treino:**
> 
> *   **Agachamento:** (3 séries de 8-12 repetições)
>     *   **Como fazer:** Pés na largura dos ombros, coluna reta, e agache como se fosse sentar em uma cadeira. Imagine que tem um banco atrás de você.
>     *   **FitNerd Dica:** Mantenha o abdômen contraído e o peso nos calcanhares.
> *   **Supino com Halteres:** (3 séries de 8-12 repetições)
>     *   **Como fazer:** Deite no banco, pés firmes no chão, e empurre os halteres para cima.
>     *   **FitNerd Dica:** Não trave os cotovelos no topo do movimento.
> *   **Remada Curvada com Halteres:** (3 séries de 8-12 repetições)
>     *   **Como fazer:** Incline o corpo para frente, mantenha a coluna reta e puxe os halteres em direção ao abdômen.
>     *   **FitNerd Dica:** Concentre-se em usar os músculos das costas para puxar o peso.
> *   **Flexão:** (3 séries até a falha ou com apoio nos joelhos)
>     *   **Como fazer:** Mãos no chão na largura dos ombros, corpo reto, e abaixe o peito em direção ao chão.
>     *   **FitNerd Dica:** Se precisar, apoie os joelhos no chão para facilitar o movimento.
> 
> **3. Core:**
> 
> *   **Prancha:** (3 séries, segurando por 30-60 segundos)
>     *   **Como fazer:** Apoie os antebraços no chão, corpo reto da cabeça aos pés.
>     *   **FitNerd Dica:** Mantenha o abdômen contraído e evite deixar o quadril cair.
> 
> **4. Relaxamento:** (5 minutos)
> 
> *   **Alongamentos:** Estique os músculos que foram trabalhados no treino (pernas, peito, costas, braços).
> 
> **Frequência:**
> 
> *   2-3 vezes por semana, com pelo menos um dia de descanso entre os treinos.
> 
> **Progressão:**
> 
> *   Aumente o peso gradualmente, à medida que se sentir mais forte.
> *   Varie os exercícios para manter o treino interessante e desafiador.
> 
> **Lembre-se:**
> 
> *   A técnica é mais importante do que a carga.
> *   Escute seu corpo e descanse quando precisar.
> *   Beba água antes, durante e depois do treino.
> *   Consulte um profissional da FitNerd para ajustar o plano às suas necessidades.
> 
> **FitNerd Palavra de especialista:**
> 
> "Este é apenas o começo! Com consistência e dedicação, você vai se surpreender com os resultados. O importante é se divertir e aproveitar o processo!"
> 
> **Observações:**
> 
> *   Adaptar o plano para diferentes níveis de condicionamento físico.
> *   Incluir fotos ou vídeos dos exercícios.
> *   Criar um senso de comunidade e pertencimento.
> *   Incentivar a interação e o feedback dos alunos.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O plano está bom, mas podemos deixá-lo ainda melhor para os iniciantes da FitNerd! Aqui estão minhas sugestões:
> 
> *   **Objetivo:**
>     *   Sugestão: "Bem-vindo(a) ao mundo dos treinos! Com este plano, você vai começar a ganhar força, queimar calorias e se sentir mais preparado(a) para as atividades do dia a dia!"
> *   **Por que Multiarticulares?**
>     *   Sugestão: "Exercícios multiarticulares são ótimos porque usam vários músculos ao mesmo tempo! Pense neles como um "super treino" que traz muitos benefícios:"
>     *   Adicionar um benefício extra: "Melhora o equilíbrio e a postura: Essencial para evitar lesões e ter mais confiança nos movimentos."
> *   **O Plano de Ataque:**
>     *   Sugestão: "Vamos fazer um circuito com 4 exercícios principais. O mais importante é fazer tudo certinho e no seu ritmo. Não tenha pressa!"
> *   **Treino:**
>     *   **Agachamento:**
>         *   Sugestão: "Pés um pouco mais afastados que a largura dos ombros, coluna reta. Desça como se fosse sentar em uma cadeira, jogando o quadril para trás."
>         *   **FitNerd Dica:** "Contraia o abdômen (como se fosse levar o umbigo para dentro) e mantenha o peso nos calcanhares para não sobrecarregar os joelhos."
>     *   **Supino com Halteres:**
>         *   Sugestão: "Deite de costas no banco, com os pés bem apoiados no chão. Segure um haltere em cada mão e empurre-os para cima."
>         *   **FitNerd Dica:** "Não estique completamente os braços no final do movimento. Deixe os cotovelos levemente dobrados para proteger as articulações."
>     *   **Remada Curvada com Halteres:**
>         *   Sugestão: "Incline o corpo para frente, mantendo as costas retas (cuidado para não curvar!). Segure um haltere em cada mão e puxe-os em direção à barriga."
>         *  **FitNerd Dica:** "Pense em usar os músculos das costas para fazer força, e não os braços. Se sentir dor na lombar, diminua o peso ou peça ajuda a um profissional."
>     *   **Flexão:**
>         *   Sugestão: "Mãos no chão, um pouco mais afastadas que a largura dos ombros. Estique as pernas e abaixe o corpo até o peito quase tocar o chão."
>         *   **FitNerd Dica:** "Se estiver difícil, apoie os joelhos no chão. O importante é manter a postura correta e fazer o movimento completo."
> *   **Core:**
>     *   **Prancha:**
>         *   Sugestão: "Apoie os antebraços no chão, com os cotovelos alinhados aos ombros. Mantenha o corpo reto, como uma prancha, da cabeça aos pés."
>         *   **FitNerd Dica:** "Contraia o abdômen e os glúteos para não deixar o quadril cair. Se sentir dor na lombar, pare e descanse."
> *   **Relaxamento:**
>     *   Sugestão: "Faça alongamentos leves para relaxar os músculos que você usou no treino. Segure cada alongamento por 20-30 segundos."
>     *   Adicionar exemplos de alongamentos: "Alongue as pernas (quadríceps, posterior da coxa), o peito, as costas e os braços."
> *   **Frequência:**
>     *   Sugestão: "Descanse pelo menos um dia entre os treinos para que seus músculos se recuperem."
> *   **Progressão:**
>     *   Sugestão: "Quando sentir que está fácil, aumente o peso dos halteres ou a dificuldade dos exercícios. Mas lembre-se: a técnica é sempre mais importante que a carga!"
> *   **Lembre-se:**
>     *   Sugestão: "Escute sempre o seu corpo e não force a barra. Se sentir dor, pare e procure um profissional da FitNerd."
> *   **FitNerd Palavra de especialista:**
>     *   Sugestão: "Lembre-se que este é só o começo da sua jornada fitness! Com dedicação e alegria, você vai alcançar seus objetivos e se sentir cada vez melhor. Conte com a FitNerd para te ajudar nessa caminhada!"
> *   **Observações:**
>     *   "Adicionar um espaço para os alunos registrarem seus pesos e repetições a cada treino."
>     *   "Criar um sistema de recompensas para motivar os alunos a continuarem treinando."
> 
> Com essas pequenas mudanças, o plano ficará ainda mais claro, amigável e motivador para os alunos iniciantes da FitNerd!


--------------------------------------------------------------
